<a href="https://colab.research.google.com/github/molonepa/pneumonia_classifier/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import packages

In [ ]:

import scipy
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import numpy as np
import matplotlib.pyplot as plt

Set path variables and print image counts

In [ ]:
DATA_PATH = 'data'

TRAIN_DIR = os.path.join(DATA_PATH, 'train')
TEST_DIR = os.path.join(DATA_PATH, 'test')
VAL_DIR = os.path.join(DATA_PATH, 'val')

train_normal_dir = os.path.join(TRAIN_DIR, 'NORMAL')
train_pneumonia_dir = os.path.join(TRAIN_DIR, 'PNEUMONIA')
test_normal_dir = os.path.join(TEST_DIR, 'NORMAL')
test_pneumonia_dir = os.path.join(TEST_DIR, 'PNEUMONIA')
val_normal_dir = os.path.join(VAL_DIR, 'NORMAL')
val_pneumonia_dir = os.path.join(VAL_DIR, 'PNEUMONIA')

print('NORMAL train images: {}'.format(len(os.listdir(train_normal_dir))))
print('PNEUMONIA train images: {}'.format(len(os.listdir(train_pneumonia_dir))))
print('total train images: {}\n--'.format(len(os.listdir(train_normal_dir)) + len(os.listdir(train_pneumonia_dir))))

print('NORMAL test images: {}'.format(len(os.listdir(test_normal_dir))))
print('PNEUMONIA test images: {}'.format(len(os.listdir(test_pneumonia_dir))))
print('total train images: {}\n--'.format(len(os.listdir(test_normal_dir)) + len(os.listdir(test_pneumonia_dir))))

print('NORMAL val images: {}'.format(len(os.listdir(val_normal_dir))))
print('PNEUMONIA val images: {}'.format(len(os.listdir(val_pneumonia_dir))))
print('total val images: {}'.format(len(os.listdir(val_normal_dir)) + len(os.listdir(val_pneumonia_dir))))

Convert images into floating point tensors using ```ImageDataGenerator``` from ```tf.keras``` and plot them


In [ ]:
# This function will plot images in the form of a grid with 1 row and 5 columns where images are placed in each column.
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img.astype('uint8'))
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
BATCH_SIZE = 64
NUM_EPOCHS = 15
IMG_HEIGHT = 500
IMG_WIDTH = 500

train_image_generator = ImageDataGenerator()
test_image_generator = ImageDataGenerator()
val_image_generator = ImageDataGenerator()

train_data_gen = train_image_generator.flow_from_directory(batch_size=BATCH_SIZE, directory=TRAIN_DIR, shuffle=True, target_size=(IMG_HEIGHT, IMG_WIDTH), class_mode='binary')
test_data_gen = test_image_generator.flow_from_directory(batch_size=BATCH_SIZE, directory=TEST_DIR, shuffle=True, target_size=(IMG_HEIGHT, IMG_WIDTH), class_mode='binary')
val_data_gen = val_image_generator.flow_from_directory(batch_size=BATCH_SIZE, directory=VAL_DIR, shuffle=True, target_size=(IMG_HEIGHT, IMG_WIDTH), class_mode='binary')

sample_training_images, _ = next(train_data_gen)

plotImages(sample_training_images[:5])

Model definition

In [ ]:
model = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1)
])
model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=['accuracy'])
model.summary()

Train

In [ ]:
history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=len(os.listdir(train_normal_dir)) + len(os.listdir(train_pneumonia_dir)) // BATCH_SIZE,
    epochs=NUM_EPOCHS,
    validation_data=val_data_gen,
    validation_steps=len(os.listdir(val_normal_dir)) + len(os.listdir(val_pneumonia_dir)) // BATCH_SIZE
)

In [ ]:
import numpy as np
for f in (np.float32, np.float64, float):
    finfo = np.finfo(f)
    print(finfo.dtype, finfo.nexp, finfo.nmant)